In [1]:
import random
import numpy as np
from math import sqrt

## Create tic-tac-toe

In [2]:
# This class saves a borad, checks and makes moves on that board.
class game:
    
    def __init__(self):
        
        # Initialize the board as a dictionary
        self.board = {}
        for i in range(1,10):
            self.board[i] = ' '
        
        # X plays the first move, we are still playing and there is no winner yet
        self.turn = 'x'
        self.playing = True
        self.winner = 'draw'
        
    # Returns all the moves possible from board
    def possible_moves(self):
        return [move for move, val in self.board.items() if val == ' ']
    
    def make_move(self, move):
        self.board[int(move)] = self.turn
        
        if self.is_over():
            self.playing  = False
        else:
            if self.turn == 'x':
                self.turn = 'o'
            else:
                self.turn = 'x'
            
    # Checks if game is over and assigns the correct winner
    def is_over(self):
        
        # Check if someone won by three in a row..
        three_in_row =  (   self.board[7] == self.board[8] == self.board[9] != ' ' 
                or self.board[7] == self.board[8] == self.board[9] != ' '
                or self.board[4] == self.board[5] == self.board[6] != ' '
                or self.board[1] == self.board[2] == self.board[3] != ' '
                or self.board[1] == self.board[4] == self.board[7] != ' '
                or self.board[2] == self.board[5] == self.board[8] != ' '
                or self.board[3] == self.board[6] == self.board[9] != ' '
                or self.board[7] == self.board[5] == self.board[3] != ' '
                or self.board[1] == self.board[5] == self.board[9] != ' ')
                         
        # Or if there are no more moves
        out_of_moves = (len(self.possible_moves()) == 0)
        
        # Assignm correct winner
        if three_in_row:
            self.winner = self.turn
            
        return three_in_row or out_of_moves

    # Print a text representation of the board
    def print_board(self):
        print('\r')
        print(self.board[7] + '|' + self.board[8] + '|' + self.board[9])
        print('-----')
        print(self.board[4] + '|' + self.board[5] + '|' + self.board[6])
        print('-----')
        print(self.board[1] + '|' + self.board[2] + '|' + self.board[3])

## Create tree node

In [3]:
# This class saves a node which keeps track of what move was made to get to it
# aswell as parents kids and other relevant data for the algorithm.
class node:      
    def __init__(self, move, parent):
        self.parent = parent 
        self.move = move
        self.children = []
        self.wins = {'x':0, 'o':0, 'draw':0}
        self.simulations = 0
        self.turn = 'draw'
        
    # Print a string representation of the node (messy for large trees).
    def __str__(self, level=0):
        res = "\t"*level+repr(self.wins['x'])+' '+ repr(self.turn)+"\n"
        for child in self.children:
            res += child.__str__(level+1)
        return res
    
    # Make child nodes with the available moves in the given game.
    def make_children(self, game):
        for move in game.possible_moves():
            self.children.append(node(move, self))

## Help functions

In [4]:
# Retuns the upper confidence tree values given C.
def calc_UCT(node, c):

    if node.simulations == 0:
        return np.inf
    win_rate = (node.wins[node.turn] + node.wins['draw']) / node.simulations
    root = c * sqrt(np.log(node.parent.simulations) / node.simulations)
    return win_rate + root

# This method returns the child with best winrate.
def calc_best_child(node):
    scores = []
    for child in node.children:
        if child.simulations == 0:
            scores.append(0)
        else:
            scores.append(child.simulations)
    return node.children[np.argmax(scores)]

## Monte carlo tree search

In [5]:
import random
import time
import copy

class MCTreeSearch:
    
    def __init__(self, c = 1.4):
        
        # create a root for the tree.
        self.c = c
        self.root = node(None, None)         
       
        # Method to pick child node based on UCT score.
    def pick_child(self, node):
        scores = []
        
        for child in node.children:
            scores.append(calc_UCT(child, self.c))
        return node.children[np.argmax(scores)]  
    
    # The select, expand, simulate, backpropogate methods
    # -------
    def select(self, game):
        
        selected_node = self.root
        while len(selected_node.children) != 0 and game.playing:
            selected_node = self.pick_child(selected_node)
            game.make_move(selected_node.move)
        return selected_node, game
    
    def expand(self, given_node, game):
        for move in game.possible_moves():
            new_node = node(move, given_node)
            new_node.turn = game.turn
            given_node.children.append(new_node)
        
    def simulate(self, node, g):
        moves = g.possible_moves()
        random.shuffle(moves)
        while g.playing:
            g.make_move(moves.pop())  
        return g.winner
            
    # Method to backpropogate and update simulation counts aswell as who won
    def backpropogate(self, winner, node):
        # Update winner and simulation
        temp_node = node
        while temp_node != None:
            temp_node.simulations += 1
            temp_node.wins[winner] = temp_node.wins[winner] + 1
            
            temp_node = temp_node.parent
    # -----
            
    # Iterate over how many games we want to play.
    def find_move(start_game, n, C = 1.4):
        
        tree = MCTreeSearch(c = C)
        selected_node = tree.root

        for i in range(n):

            game = copy.deepcopy(start_game)
            
            # Select
            selected_node, game = tree.select(game)

            # Expand
            if game.playing:
                tree.expand(selected_node, game)
                selected_node = random.choice(selected_node.children)
                game.make_move(selected_node.move)
                
                # Simulate
                winner = tree.simulate(selected_node, game)
                
            else:
                winner = game.winner
            
            # Backpropogate
            tree.backpropogate(winner, selected_node)
            
        return calc_best_child(tree.root).move

In [6]:
# Method for a to play againts a random oponent.
def play_self_game(i, c):
    g = game()
    while g.playing:
        move = MCTreeSearch.find_move(g, i, c)        
        g.make_move(move)

        if not g.playing:
            break   
            
        move = MCTreeSearch.find_move(g, i, c)        
        g.make_move(move)

    return g.winner

In [7]:
# Method for a to play againts a random oponent.
def play_random_game(i, c):
    g = game()
    while g.playing:
        move = MCTreeSearch.find_move(g, i, c)        
        g.make_move(move)

        if not g.playing:
            break   
            
        random_move = np.random.choice(g.possible_moves())
        g.make_move(random_move)

    return g.winner

In [8]:
def play_random_game_reverse(i, c):
    g = game()
    while g.playing:
        random_move = np.random.choice(g.possible_moves())
        g.make_move(random_move)


        if not g.playing:
            break   
            
        move = MCTreeSearch.find_move(g, i, c)        
        g.make_move(move)    


    return g.winner

In [9]:
def play_game(i):

    # Start game, begin at root of tree.
    g = game()

    # While no one has won...
    while g.playing:
        # Make the best move by picking the best child!
        move = MCTreeSearch.find_move(g, i)
        g.make_move(move)
        if not g.playing: # Break if someone won.
            break

        # Print board and wait for user to input next move.
        g.print_board()
        player_input = input()
        g.make_move(player_input)


    # Print end of board
    g.print_board()

## Train and test our model.

In [10]:
wins = 0
losses = 0
for i in range(50):
    if play_random_game(500,2) == 'x':
        wins += 1
    if play_random_game(500,2) == 'o':
        losses += 1

# Print wins, draws, and losses.
print("Games against random opponent.")
print("win percentage:",wins/i)
print("loss percentage:",losses/i)
print("draw percentage:", (i-wins-losses)/i)

Games against random opponent.
win percentage: 0.8367346938775511
loss percentage: 0.0
draw percentage: 0.16326530612244897


In [11]:
wins = 0
losses = 0
for i in range(50):
    if play_random_game_reverse(500,2) == 'x':
        wins += 1
    if play_random_game_reverse(500,2) == 'o':
        losses += 1

# Print wins, draws, and losses.
print("Games against random oponent staring second.")
print("win percentage:",wins/i)
print("loss percentage:",losses/i)
print("draw percentage:", (i-wins-losses)/i)

Games against random oponent staring second.
win percentage: 0.0
loss percentage: 0.7346938775510204
draw percentage: 0.2653061224489796


In [12]:
wins = 0
losses = 0
for i in range(50):
    if play_self_game(500,2) == 'x':
        wins += 1
    if play_self_game(500,2) == 'o':
        losses += 1

# Print wins, draws, and losses.
print("Games against itself.")
print("win percentage:",wins/i)
print("loss percentage:",losses/i)
print("draw percentage:", (i-wins-losses)/i)

Games against itself.
win percentage: 0.0
loss percentage: 0.0
draw percentage: 1.0


Games against brute force method was removed here to avoid having to rename the classes and filling the document with to much code

In [13]:
# Run to play game against user.
play_game(500)


 | | 
-----
 |x| 
-----
 | | 
1

 | | 
-----
x|x| 
-----
o| | 
6

 | | 
-----
x|x|o
-----
o| |x
7

o| | 
-----
x|x|o
-----
o|x|x
8

o|o|x
-----
x|x|o
-----
o|x|x
